In [1]:
x1 = 'SEND'
x2 = 'MORE'
answer = 'MONEY'

In [64]:
def tupleMaker(x1, x2, answer):
    x1_chars = [i for i in x1]
    x2_chars = [i for i in x2]
    answer_chars = [i for i in answer]
    set_chars = x1_chars + x2_chars + answer_chars
    tupple_chars = tuple(set(set_chars))    
    return tupple_chars

In [65]:
tuple_chars = tupleMaker(x1, x2, answer)

In [36]:
domain = [i for i in range(10)]

In [69]:
def checkIfCorrect(x1, x2, answer,dict_chars):
    num1 = int(''.join([str(dict_chars[i]) for i in x1]))
    num2 = int(''.join([str(dict_chars[i]) for i in x2]))
    ans_int = int(''.join([str(dict_chars[i]) for i in answer]))
    sum_ans = sum([num1, num2])
    if ans_int == sum_ans:
        return (True,num1,num2,sum_ans)
    else:
        return (False,num1,num2,sum_ans)

In [61]:
checkIfCorrect(x1,x2,{'R': 8, 'E': 5, 'S': 9, 'M': 1, 'N': 6, 'O': 0, 'D': 7, 'Y':2})

(True, 9567, 1085, 10652)

### Brute Force Approach

In [70]:
import itertools
def solve(tuple_chars, domain, x1, x2, answer):
    digits = range(10)
    for perm in itertools.permutations(domain, len(tuple_chars)):
        sol = dict(zip(tuple_chars, perm))
        if sol[x1[0]] == 0 or sol[x2[0]] == 0:
            continue
        result = checkIfCorrect(x1,x2,answer,sol)
        if result[0]:
            return result[1], result[2], result[3]

In [71]:
solve(tuple_chars, domain, x1, x2, answer)

(9567, 1085, 10652)

In [73]:
x1 = 'BASE'
x2 = 'BALL'
answer = 'GAMES'
tuple_chars = tupleMaker(x1,x2,answer)
solve(tuple_chars, domain, x1,x2,answer)

(2461, 2455, 4916)

In [78]:
x1 = 'CROSS'
x2 = 'ROADS'
answer = 'DANGER'
tuple_chars = tupleMaker(x1,x2,answer)
solve(tuple_chars, domain, x1,x2,answer)

(96233, 62513, 158746)

### Proper Approach

In [97]:
# Typing is a type checker for python
# Generic -> Abstract base class for generic types
# TypeVar -> Primarily exists for static type checking (Name assigned with the type that is acceptable for that variable)
# List -> declares what type the list is able to hold
# Dict -> declares type of key and type of value specifically 
# Optional -> optional variable concept

# abc  is Abstract Base Classes Library of python
# ABC -> an abstract class can be generated by simply deriving from ABC
# abstractmethod -> A decorator indicating abstract method
from typing import Generic, TypeVar, List, Dict, Optional
from abc import ABC, abstractmethod

V = TypeVar('V') #Type of the variable
D = TypeVar('D') #Type of the domain

class Constraint(Generic[V, D], ABC):
    def __init__(self, variables: List[V]) -> None:
        self.variables = variables

    @abstractmethod
    def satisfied(self, assignment: Dict[V, D]) -> bool:
        ...

In [118]:
class CSP(Generic[V, D]):
    def __init__(self, variables: List[V], domains: Dict[V, List[D]]) -> None:
        self.variables: List[V] = variables # variables to be constrained
        self.domains: Dict[V, List[D]] = domains # domain of each variable
        self.constraints: Dict[V, List[Constraint[V, D]]] = {}
        for variable in self.variables:
            self.constraints[variable] = []
            if variable not in self.domains:
                raise LookupError("Every variable should have a domain assigned to it.")

    def add_constraint(self, constraint: Constraint[V, D]) -> None:
        for variable in constraint.variables:
            if variable not in self.variables:
                raise LookupError("Variable in constraint not in CSP")
            else:
                self.constraints[variable].append(constraint)

    def consistent(self, variable: V, assignment: Dict[V, D]) -> bool:
        for constraint in self.constraints[variable]:
            if not constraint.satisfied(assignment):
                return False
        return True

    def backtracking_search(self, assignment: Dict[V, D] = {}) -> Optional[Dict[V, D]]:
        if len(assignment) == len(self.variables):
            return assignment

        unassigned: List[V] = [v for v in self.variables if v not in assignment]

        first: V = unassigned[0]
        for value in self.domains[first]:
            local_assignment = assignment.copy()
            local_assignment[first] = value
            if self.consistent(first, local_assignment):
                result: Optional[Dict[V, D]] = self.backtracking_search(local_assignment)
                if result is not None:
                    return result
        return None

In [119]:
class SendMoreMoneyConstraint(Constraint[str, int]):
    def __init__(self, letters: List[str])->None:
        super().__init__(letters)
        self.letters: List[str] = letters
    
    def satisfied(self, assignment: Dict[str, int]) -> bool:
        if len(set(assignment.values())) < len(assignment):
            return False

        if len(assignment) == len(self.letters):
            s: int = assignment["S"]
            e: int = assignment["E"]
            n: int = assignment["N"]
            d: int = assignment["D"]
            m: int = assignment["M"]
            o: int = assignment["O"]
            r: int = assignment["R"]
            y: int = assignment["Y"]
            send: int = s * 1000 + e * 100 + n * 10 + d
            more: int = m * 1000 + o * 100 + r * 10 + e
            money: int = m * 10000 + o * 1000 + n * 100 + e * 10 + y
            return send + more == money
        return True 

In [120]:
letters: List[str] = ["S", "E", "N", "D", "M", "O", "R", "Y"]
possible_digits: Dict[str, List[int]] = {}
for letter in letters:
    possible_digits[letter] = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
possible_digits["M"] = [1]  # so we don't get answers starting with a 0
csp: CSP[str, int] = CSP(letters, possible_digits)
csp.add_constraint(SendMoreMoneyConstraint(letters))
solution: Optional[Dict[str, int]] = csp.backtracking_search()
if solution is None:
    print("No solution found!")
else:
    print(solution)

{'S': 9, 'E': 5, 'N': 6, 'D': 7, 'M': 1, 'O': 0, 'R': 8, 'Y': 2}
